In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchaudio
import requests
import matplotlib.pyplot as plt
from pydub import AudioSegment
import os
import pandas as pd
import numpy as np
import time
torchaudio.USE_SOUNDFILE_LEGACY_INTERFACE = False

C:\Users\Admin\anaconda3\lib\site-packages\torchaudio\extension\extension.py:14: UserWarning: torchaudio C++ extension is not available.
  warnings.warn('torchaudio C++ extension is not available.')
C:\Users\Admin\anaconda3\lib\site-packages\torchaudio\backend\utils.py:64: UserWarning: The interface of "soundfile" backend is planned to change in 0.8.0 to match that of "sox_io" backend and the current interface will be removed in 0.9.0. To use the new interface, do `torchaudio.USE_SOUNDFILE_LEGACY_INTERFACE = False` before setting the backend to "soundfile". Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  'The interface of "soundfile" backend is planned to change in 0.8.0 to '


In [4]:
import csv
from sklearn.model_selection import train_test_split

with open('C:\\Users\\Admin\\Downloads\\combine_all_v2.csv', mode='r', encoding='utf-8', newline='') as file:
    reader = csv.reader(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    data = [line for line in reader]
    
head = data[0]
train_data, test_data = train_test_split(data[1:], train_size = 0.8)
with open('train.csv', mode='w', encoding='utf-8', newline='') as file:
    writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(head)
    for line in train_data:
        writer.writerow(line)
        
with open('test.csv', mode='w', encoding='utf-8', newline='') as file:
    writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(head)
    for line in test_data:
        writer.writerow(line)

In [2]:
data = pd.read_csv("train.csv")
print(data.shape)
user_ids = data.user_id.unique()
song_ids = data.song_id.unique()
num_users = len(user_ids)
num_songs = len(song_ids)
user_ids_to_idx = dict(zip(user_ids, range(num_users)))
song_ids_to_idx = dict(zip(song_ids, range(num_songs)))

Y = np.zeros((num_users, num_songs))

for i in range(data.shape[0]):
    user_idx = user_ids_to_idx[data.user_id[i]]
    song_idx = song_ids_to_idx[data.song_id[i]]
    Y[user_idx][song_idx] = int(data.listen_count[i])
    

    

(86425, 9)


In [6]:
print(Y.shape)
print(Y.T.shape)

(53196, 9244)
(9244, 53196)


In [28]:
print(Y[[1,3,5],[2,4,6]])
print(99//5)
print(19*5)

[0. 0. 0.]
19
95


In [3]:
class RecSys(nn.Module):
    def __init__(self, num_users, num_songs, embedding_size):
        super().__init__()
        self.num_users = num_users
        self.num_songs = num_songs
        self.embedding_size = embedding_size
        self.user_encode = nn.Sequential(
            nn.Linear(num_songs, embedding_size, bias=False),
            nn.Linear(embedding_size, embedding_size),
            nn.ReLU(),
            nn.Linear(embedding_size, embedding_size),
            nn.ReLU(),
            nn.Linear(embedding_size, embedding_size)
        )
        self.song_encode = nn.Sequential(
            nn.Linear(num_users, embedding_size, bias=False),
            nn.Linear(embedding_size, embedding_size),
            nn.ReLU(),
            nn.Linear(embedding_size, embedding_size),
            nn.ReLU(),
            nn.Linear(embedding_size, embedding_size)
        )
        self.cosine = nn.CosineSimilarity()

    def forward(self, user, song):
        user = torch.tensor(user).cuda().float()
        song = torch.tensor(song).cuda().float()
        p = self.user_encode(user)
        q = self.song_encode(song)
        out = self.cosine(p, q)
        return p, q, out

In [20]:
model = RecSys(num_users=num_users, num_songs=num_songs, embedding_size=256).cuda().float()
p, q, out = model(Y[:2],Y.T[:2])
print(p.shape,q.shape)
print(out)

torch.Size([2, 512]) torch.Size([2, 512])
tensor([-0.0203, -0.0111], device='cuda:0', grad_fn=<DivBackward0>)


In [4]:
model = RecSys(num_users=num_users, num_songs=num_songs, embedding_size=512).cuda().float()

def train(model, Y, num_epochs=100, batch_size=128, lr=0.001):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    num_batch_user = num_users//batch_size
    num_batch_song = num_songs//batch_size
    for epoch in range(1, num_epochs+1):
        model.train()
        losses = []
        user_idx = list(range(num_users))
        np.random.shuffle(user_idx)
        song_idx = list(range(num_songs))
        np.random.shuffle(song_idx)
        c1 = time.time()
        for i in range(num_batch_user):
            if (i+1)*batch_size > num_users:
                u_i = user_idx[i*batch_size: num_users]
            else:
                u_i = user_idx[i*batch_size: (i+1)*batch_size]
            
            user_batch = Y[u_i]
            for j in range(num_batch_song):    
                if (j+1)*batch_size > num_songs:
                    v_j = song_idx[j*batch_size: num_songs]
                else:
                    v_j = song_idx[j*batch_size: (j+1)*batch_size]
                    
                song_batch = Y.T[v_j]
                
                p, q, out = model(user_batch, song_batch)
                target = torch.tensor(Y[u_i, v_j]).cuda().float()
                loss = criterion(out, target)
                losses.append(loss)
                loss.backward()
                optimizer.step()
                
        c2 = time.time()
        print('epoch: {}, time: {}s, loss: {}'.format(epoch, c2-c1, sum(losses)/len(losses)))
        losses = []
        
                
train(model, Y)

KeyboardInterrupt: 

In [ ]:
import os
from os import listdir
from os.path import isfile, isdir, join
import csv
import eyed3
import pandas as pd
import numpy as np

title = pd.read_csv("C:\\Users\\Admin\\Downloads\\combine_all_v2.csv").title.unique()
directory = 'C:/Users/Admin/Downloads/fma_medium'
full = listdir(directory)
d = []

for i in full:
    if isdir(join(directory, i)):
        d.append(i)
        
for direc in d:
    file = listdir(join(directory, direc))
    for f in file:
        audio = eyed3.load(join(directory, direc, f))
        t = audio.tag.title
        if t in title:
            

In [ ]:
waveform, sample_rate = torchaudio.load('test.wav')
print(waveform.shape)
print(sample_rate)
plt.figure()
plt.plot(waveform.t().numpy())
specgram = torchaudio.transforms.Spectrogram()(waveform)

print("Shape of spectrogram: {}".format(specgram.size()))

plt.figure()
plt.imshow(specgram.log2()[0,:,:].numpy(), cmap='gray')

In [ ]:
import IPython.display as ipd

url = "https://pytorch.org/tutorials/_static/img/steam-train-whistle-daniel_simon-converted-from-mp3.wav"
r = requests.get(url)

with open('steam-train-whistle-daniel_simon-converted-from-mp3.wav', 'wb') as f:
    f.write(r.content)

filename = "steam-train-whistle-daniel_simon-converted-from-mp3.wav"
waveform, sample_rate = torchaudio.load(filename)

print("Shape of waveform: {}".format(waveform.size()))
print("Sample rate of waveform: {}".format(sample_rate))

plt.figure()
plt.plot(waveform.t().numpy())

new_sample_rate = 8000
transform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=new_sample_rate)
transformed = transform(waveform)

ipd.Audio(transformed.numpy(), rate=new_sample_rate)
plt.figure()
plt.plot(transformed.t().numpy())
print(transformed.shape)